Deep Learning for Chord Recognition
# McGill Billboard Dataset

This notebook implements a deep learning approach to chord recognition using the McGill Billboard dataset. The goal is to predict chord labels from audio chroma features.

## Table of Contents
1. Introduction and Problem Description
2. Data Exploration (EDA)
3. Data Preparation
4. Model Development
5. Training and Evaluation
6. Results and Analysis
7. Conclusion and Future Work

# 1. Introduction and Problem Description

In this project, we implement a deep learning solution for automatic chord recognition from audio. Chord recognition is a fundamental task in music information retrieval (MIR) with applications in music education, transcription, recommendation systems, and musicological analysis. 

We aim to develop and compare multiple neural network architectures for identifying chords from audio features, specifically using chroma representations derived from music audio files. 

Our approach includes implementing memory-efficient data handling techniques to process a large dataset, and incorporating temporal context to improve chord recognition accuracy.

## Dataset Description

We'll be using the McGill Billboard dataset, which contains:
- Chroma features extracted from popular music recordings
- Time-aligned chord labels
- 890 songs spanning several decades of popular music

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import f_oneway
import seaborn as sns
import os
import glob
from pathlib import Path
import re
from collections import Counter
import pickle
from tqdm import tqdm
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten, Input, BatchNormalization, Bidirectional, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.colors as mcolors
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Constants
EPOCHS = 30
BATCH_SIZE = 256
SEQUENCE_LENGTH = 15  # Number of frames to use for context window
STEP_SIZE = 1  # Step size for creating sequences
VALIDATION_SPLIT = 0.2
TEST_SPLIT = 0.1

In [ ]:
# Set the paths
data_dir = Path('data/McGill-Billboard')
chordino_dir = data_dir / 'chordino'
lab_dir = data_dir / 'lab'
annotations_dir = data_dir / 'annotations'
index_path = data_dir / 'index.csv'

In [ ]:
# Set random seed for reproducibility
np.random.seed(33)
tf.random.set_seed(33)
random.seed(33)

In [ ]:
# Create directories for models and results
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)


# 2. Data Exploration (EDA)

Let's first explore the dataset to understand its structure and content.

In [ ]:
# Load the dataset index
index_df = pd.read_csv(index_path)
print(f"Total entries in index: {len(index_df)}")
print(f"Entries with complete data: {index_df['title'].notna().sum()}")

In [ ]:
# Drop entries with missing/incomplete/unavailable data
index_df = index_df.dropna(subset=['title'])
print(f'Total Entries after cleaning: {len(index_df)}')

In [ ]:
# Display basic information
print(f"Number of unique songs: {index_df['title'].nunique()}")
print(f"Number of unique artists: {index_df['artist'].nunique()}")

In [ ]:
# Convert chart_date to datetime
index_df['chart_date'] = pd.to_datetime(index_df['chart_date'])

In [ ]:
# Extract year and decade
index_df['year'] = index_df['chart_date'].dt.year
index_df['decade'] = (index_df['year'] // 10) * 10

In [ ]:
# Plot distribution by decade
plt.figure(figsize=(10, 6))
decade_counts = index_df['decade'].value_counts().sort_index()
decade_counts.plot(kind='bar', color='skyblue')
plt.title('Number of Songs by Decade')
plt.xlabel('Decade')
plt.ylabel('Count')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Check available directories
song_ids = [d.name for d in chordino_dir.iterdir() if d.is_dir()]
print(f"Number of songs with chroma features: {len(song_ids)}")

In [ ]:
# Check if the same songs have lab files
lab_ids = [d.name for d in lab_dir.iterdir() if d.is_dir()]
print(f"Number of songs with lab files: {len(lab_ids)}")

In [ ]:
# Find songs that have both chroma and labels
common_ids = set(song_ids).intersection(set(lab_ids))
print(f"Number of songs with both features and labels: {len(common_ids)}")

## Exploring a Random Example

Let's examine a random song from the dataset to understand the structure of the chroma features and chord labels.

In [ ]:
# Choose a random example
example_id = list(common_ids)[random.randint(0, len(common_ids) - 1)]

In [ ]:
# Load chroma features for the example song
chroma_path = chordino_dir / example_id / 'bothchroma.csv'
tuning_path = chordino_dir / example_id / 'tuning.csv'

In [ ]:
if chroma_path.exists() and tuning_path.exists():
    chroma_data = pd.read_csv(chroma_path, header=None)
    tuning_data = pd.read_csv(tuning_path, header=None)
    
    print(f"\nChroma shape for song {example_id}: {chroma_data.shape}")
    print(f"Tuning shape for song {example_id}: {tuning_data.shape}")
    
    # Display a sample of the chroma data
    print("\nSample of chroma data:")
    print(chroma_data.head())
    
    # Extract chroma values (assuming columns 2-13 contain the chroma values)
    chroma_values = chroma_data.iloc[:, 2:14].values
    if chroma_values.shape[1] != 12:
        # If the format is different, try to find the 12 consecutive columns with numeric data
        for i in range(len(chroma_data.columns) - 12 + 1):
            test_values = chroma_data.iloc[:, i:i+12].values
            if np.issubdtype(test_values.dtype, np.number):
                chroma_values = test_values
                print(f"Found chroma values in columns {i} to {i+11}")
                break
    
    # Plot chroma data
    plt.figure(figsize=(12, 6))
    plt.subplot(211)
    sns.heatmap(chroma_values[:100, :].T, cmap='coolwarm', cbar=True)
    plt.title(f'Chroma Feature Heatmap for First 100 Frames - Song {example_id}')
    plt.xlabel('Time frames')
    plt.ylabel('Pitch Class')
    
    # Add pitch class labels on the y-axis
    pitch_classes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    plt.yticks(np.arange(12) + 0.5, pitch_classes)
    
    # Load chord labels for the example song
    lab_path = lab_dir / example_id / "full.lab"
    if lab_path.exists():
        # Read the lab file (tab-separated with no header)
        lab_data = pd.read_csv(lab_path, sep='\t', header=None, names=['start_time', 'end_time', 'chord'])
        
        print(f"\nNumber of chord segments for song {example_id}: {len(lab_data)}")
        print("\nSample of chord labels:")
        print(lab_data.head())
        
        # Count the unique chords in this song
        print(f"\nNumber of unique chords in song {example_id}: {lab_data['chord'].nunique()}")
        print("\nMost common chords:")
        print(lab_data['chord'].value_counts().head(10))
        
        # Plot chord durations
        plt.subplot(212)
        chord_durations = lab_data['end_time'] - lab_data['start_time']
        plt.hist(chord_durations, bins=30, alpha=0.7, color='teal')
        plt.title('Chord Duration Distribution')
        plt.xlabel('Duration (seconds)')
        plt.ylabel('Frequency')
        plt.tight_layout()
        plt.show()
        
        # Plot most common chords
        plt.figure(figsize=(12, 6))
        top_chords = lab_data['chord'].value_counts().head(15)
        top_chords.plot(kind='bar', color='coral')
        plt.title(f'Most Common Chords in Song {example_id}')
        plt.xlabel('Chord')
        plt.ylabel('Count')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print(f"Lab file not found for song {example_id}")
else:
    print(f"Chroma or tuning data not found for song {example_id}")

## Analyzing Chord Distribution Across the Dataset

In [ ]:
def get_chord_distribution(song_ids, top_n=20):
    """Get the distribution of chords across all songs"""
    all_chords = Counter()
    processed_songs = 0
    
    for song_id in tqdm(song_ids, desc="Analyzing chord distribution"):
        lab_path = lab_dir / song_id / "full.lab"
        
        if lab_path.exists():
            # Load and count chord labels
            lab_data = pd.read_csv(lab_path, sep='\t', header=None, names=['start_time', 'end_time', 'chord'])
            chord_counts = lab_data['chord'].value_counts().to_dict()
            
            # Add to overall counter
            all_chords.update(chord_counts)
            processed_songs += 1
    
    print(f"Processed {processed_songs} songs for chord distribution")
    return all_chords

In [ ]:
# Analyze chord distribution across a sample of songs (limit to 100 for speed)
chord_distribution = get_chord_distribution(list(common_ids)[:100])

In [ ]:
# Plot top chords
plt.figure(figsize=(14, 8))
top_chords = pd.Series(dict(chord_distribution.most_common(30)))
top_chords.plot(kind='bar', color='lightseagreen')
plt.title('Most Common Chords Across the Dataset (Top 30)')
plt.xlabel('Chord')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Print statistics
total_chords = sum(chord_distribution.values())
unique_chords = len(chord_distribution)
print(f"Total chord occurrences: {total_chords}")
print(f"Number of unique chords: {unique_chords}")

In [ ]:
# How much of the dataset is covered by the top N chords?
def coverage_analysis(chord_distribution, tops=[10, 20, 50, 100, 200]):
    total = sum(chord_distribution.values())
    for n in tops:
        top_n = sum(dict(chord_distribution.most_common(n)).values())
        print(f"Top {n} chords cover {top_n/total*100:.2f}% of all chord occurrences")

In [ ]:
coverage_analysis(chord_distribution)


# 3. Data Preparation

Now let's implement memory-efficient data handling techniques to process the large dataset.

In [ ]:
def normalize_chroma_tuning(chroma_array, song_id):
    """Normalize chroma features based on tuning information"""
    tuning_path = chordino_dir / song_id / 'tuning.csv'
    
    if not tuning_path.exists():
        return chroma_array  # Can't normalize, return original
    
    try:
        # Load tuning information
        tuning_data = pd.read_csv(tuning_path, header=None)
        tuning_frequency = float(tuning_data.iloc[0, 3])  # Assuming 4th column has the frequency
        
        # Calculate tuning deviation in semitones
        # A4 = 440Hz in standard tuning
        tuning_deviation = 12 * np.log2(tuning_frequency / 440.0)
        
        # No need to adjust if very close to standard tuning
        if abs(tuning_deviation) < 0.01:
            return chroma_array
            
        # Convert deviation to circular shift (positive means shift right)
        # We negate because if tuning is higher, we need to shift chroma left
        shift_amount = -tuning_deviation
        
        # Apply circular shift to each chroma vector
        normalized_chroma = np.zeros_like(chroma_array)
        for i in range(len(chroma_array)):
            # Use interpolation for fractional shifts
            for j in range(12):
                # Calculate the fractional bin position after shifting
                bin_pos = (j + shift_amount) % 12
                bin_idx = int(bin_pos)
                bin_frac = bin_pos - bin_idx
                
                # Distribute energy between adjacent bins (linear interpolation)
                normalized_chroma[i, bin_idx % 12] += chroma_array[i, j] * (1 - bin_frac)
                normalized_chroma[i, (bin_idx + 1) % 12] += chroma_array[i, j] * bin_frac
        
        return normalized_chroma
        
    except Exception as e:
        print(f"Error normalizing chroma for song {song_id}: {e}")
        return chroma_array  # Return original on error

def load_chroma(song_id):
    """Load chroma features for a given song ID with robust error handling"""
    chroma_path = chordino_dir / song_id / 'bothchroma.csv'
    
    if not chroma_path.exists():
        return None
    
    try:
        # Read the file
        data = pd.read_csv(chroma_path, header=None)
        
        # Figure out which columns contain the chroma data (should be 12 consecutive numeric columns)
        # Start by converting all to numeric, with errors='coerce' to handle non-numeric values
        numeric_data = data.apply(pd.to_numeric, errors='coerce')
        
        # Find the column indices where most values are not NaN (these should be our chroma columns)
        valid_columns = numeric_data.count() > 0.8 * len(numeric_data)
        valid_column_indices = valid_columns[valid_columns].index.tolist()
        
        # Find a consecutive sequence of 12 columns (for 12 pitch classes)
        chroma_start = None
        for i in range(len(valid_column_indices) - 11):
            if valid_column_indices[i:i+12] == list(range(valid_column_indices[i], valid_column_indices[i] + 12)):
                chroma_start = valid_column_indices[i]
                break
        
        if chroma_start is None:
            # If no perfect consecutive sequence, try to find the best 12 consecutive columns
            for i in range(len(data.columns) - 11):
                test_cols = data.iloc[:, i:i+12]
                if test_cols.apply(pd.to_numeric, errors='coerce').count().min() > 0.8 * len(data):
                    chroma_start = i
                    break
        
        if chroma_start is None:
            # If still no luck, just take columns 2-13 (common format)
            chroma_start = 2
        
        # Extract and convert the chroma values
        chroma_values = data.iloc[:, chroma_start:chroma_start+12].apply(pd.to_numeric, errors='coerce').values
        
        # Replace any remaining NaNs with zeros
        chroma_values = np.nan_to_num(chroma_values)
        
        # Apply tuning normalization
        chroma_values = normalize_chroma_tuning(chroma_values, song_id)
        
        return chroma_values.astype(np.float32)
    
    except Exception as e:
        print(f"Error loading chroma for song {song_id}: {e}")
        return None

def load_chord_labels(song_id):
    """Load chord labels for a given song ID"""
    lab_path = lab_dir / song_id / "full.lab"
    
    if not lab_path.exists():
        return None
    
    try:
        # Load the chord labels
        lab_data = pd.read_csv(lab_path, sep='\t', header=None, names=['start_time', 'end_time', 'chord'])
        return lab_data
    except Exception as e:
        print(f"Error loading chord labels for song {song_id}: {e}")
        return None

def align_chroma_with_chords(chroma_array, chord_data, chroma_hop_size=0.01):
    """Align chroma features with chord labels"""
    # Generate timestamps for each chroma frame
    chroma_times = np.arange(len(chroma_array)) * chroma_hop_size
    
    aligned_chords = []
    
    for i, time in enumerate(chroma_times):
        # Find the chord label for this time point
        matching_chords = chord_data[(chord_data['start_time'] <= time) & (chord_data['end_time'] > time)]
        
        if len(matching_chords) > 0:
            chord = matching_chords.iloc[0]['chord']
        else:
            chord = "N"  # No chord (silence or undefined)
        
        aligned_chords.append(chord)
    
    return aligned_chords

def create_sequences(features, labels, seq_length=15, step=1):
    """Create sequences for temporal context"""
    X, y = [], []
    for i in range(0, len(features) - seq_length + 1, step):
        X.append(features[i:i+seq_length])
        y.append(labels[i+seq_length-1])  # Target is the label of the last frame in the sequence
    
    return np.array(X), np.array(y)

In [ ]:
# Analyze tuning distribution across the dataset
def analyze_tuning_distribution():
    """Analyze the distribution of tuning frequencies across the dataset"""
    tuning_frequencies = []
    song_ids = []
    
    for song_id in tqdm(common_ids, desc="Analyzing tuning"):
        tuning_path = chordino_dir / song_id / 'tuning.csv'
        if tuning_path.exists():
            try:
                tuning_data = pd.read_csv(tuning_path, header=None)
                tuning_freq = float(tuning_data.iloc[0, 3])
                tuning_frequencies.append(tuning_freq)
                song_ids.append(song_id)
            except:
                pass
    
    # Calculate deviation in cents (100 cents = 1 semitone)
    tuning_deviations = [1200 * np.log2(freq / 440.0) for freq in tuning_frequencies]
    
    # Plot distribution
    plt.figure(figsize=(12, 6))
    plt.hist(tuning_deviations, bins=50, color='teal', alpha=0.7)
    plt.axvline(x=0, color='red', linestyle='--', label='A = 440 Hz')
    plt.xlabel('Tuning Deviation (cents)')
    plt.ylabel('Number of Songs')
    plt.title('Distribution of Tuning Deviations in the Dataset')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Plot histogram of absolute deviations
    plt.figure(figsize=(10, 6))
    plt.hist([abs(dev) for dev in tuning_deviations], bins=30, color='coral', alpha=0.7)
    plt.xlabel('Absolute Tuning Deviation (cents)')
    plt.ylabel('Number of Songs')
    plt.title('Distribution of Absolute Tuning Deviations')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"Average tuning frequency: {np.mean(tuning_frequencies):.2f} Hz")
    print(f"Median tuning frequency: {np.median(tuning_frequencies):.2f} Hz")
    print(f"Minimum tuning frequency: {np.min(tuning_frequencies):.2f} Hz")
    print(f"Maximum tuning frequency: {np.max(tuning_frequencies):.2f} Hz")
    print(f"Standard deviation: {np.std(tuning_frequencies):.2f} Hz")
    
    # Calculate percentage of songs with significant tuning deviations
    significant_deviation_count = sum(1 for dev in tuning_deviations if abs(dev) > 30)
    print(f"Songs with significant tuning deviation (>30 cents): {significant_deviation_count} ({significant_deviation_count/len(tuning_frequencies)*100:.1f}%)")
    
    # Identify outliers
    outliers = [(song_id, freq, dev) for song_id, freq, dev in 
                zip(song_ids, tuning_frequencies, tuning_deviations) 
                if abs(dev) > 50]  # More than 50 cents (half semitone) deviation
    
    if outliers:
        print("\nSignificant tuning deviations (>50 cents):")
        for song_id, freq, dev in sorted(outliers, key=lambda x: abs(x[2]), reverse=True)[:10]:
            print(f"Song {song_id}: {freq:.2f} Hz ({dev:.1f} cents)")
    
    return tuning_frequencies, tuning_deviations

In [ ]:
# Visualize the effect of tuning normalization on a specific example
def visualize_tuning_normalization(song_id=None):
    """Visualize the effect of tuning normalization on chroma features"""
    if song_id is None:
        # Find a song with significant tuning deviation
        for s_id in common_ids:
            tuning_path = chordino_dir / s_id / 'tuning.csv'
            if tuning_path.exists():
                try:
                    tuning_data = pd.read_csv(tuning_path, header=None)
                    tuning_freq = float(tuning_data.iloc[0, 3])
                    deviation = 1200 * np.log2(tuning_freq / 440.0)
                    if abs(deviation) > 30:  # More than 30 cents deviation
                        song_id = s_id
                        break
                except:
                    continue
    
    if song_id is None:
        print("Could not find a song with significant tuning deviation for visualization")
        return
    
    # Load the chroma features without normalization
    chroma_path = chordino_dir / song_id / 'bothchroma.csv'
    if not chroma_path.exists():
        print(f"Chroma file not found for song {song_id}")
        return
    
    try:
        # Read the file
        data = pd.read_csv(chroma_path, header=None)
        
        # Figure out which columns contain the chroma data
        numeric_data = data.apply(pd.to_numeric, errors='coerce')
        valid_columns = numeric_data.count() > 0.8 * len(numeric_data)
        valid_column_indices = valid_columns[valid_columns].index.tolist()
        
        # Find chroma columns
        chroma_start = None
        for i in range(len(valid_column_indices) - 11):
            if valid_column_indices[i:i+12] == list(range(valid_column_indices[i], valid_column_indices[i] + 12)):
                chroma_start = valid_column_indices[i]
                break
        
        if chroma_start is None:
            for i in range(len(data.columns) - 11):
                test_cols = data.iloc[:, i:i+12]
                if test_cols.apply(pd.to_numeric, errors='coerce').count().min() > 0.8 * len(data):
                    chroma_start = i
                    break
        
        if chroma_start is None:
            chroma_start = 2
        
        # Extract raw chroma values
        raw_chroma = data.iloc[:, chroma_start:chroma_start+12].apply(pd.to_numeric, errors='coerce').values
        raw_chroma = np.nan_to_num(raw_chroma)
        
        # Get the normalized chroma
        normalized_chroma = normalize_chroma_tuning(raw_chroma, song_id)
        
        # Get tuning information
        tuning_path = chordino_dir / song_id / 'tuning.csv'
        tuning_data = pd.read_csv(tuning_path, header=None)
        tuning_freq = float(tuning_data.iloc[0, 3])
        deviation = 12 * np.log2(tuning_freq / 440.0)
        
        # Visualize both raw and normalized chroma
        plt.figure(figsize=(15, 10))
        
        # Plot raw chroma
        plt.subplot(211)
        frame_range = min(100, len(raw_chroma))  # Show first 100 frames or less
        sns.heatmap(raw_chroma[:frame_range].T, cmap='coolwarm')
        plt.title(f'Raw Chroma Features - Song {song_id} (Tuning: {tuning_freq:.1f} Hz, {deviation*100:.1f} cents off standard)')
        plt.xlabel('Time Frames')
        plt.ylabel('Pitch Class')
        plt.yticks(np.arange(12) + 0.5, ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'])
        
        # Plot normalized chroma
        plt.subplot(212)
        sns.heatmap(normalized_chroma[:frame_range].T, cmap='coolwarm')
        plt.title('Normalized Chroma Features (Tuning Correction Applied)')
        plt.xlabel('Time Frames')
        plt.ylabel('Pitch Class')
        plt.yticks(np.arange(12) + 0.5, ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'])
        
        plt.tight_layout()
        plt.show()
        
        return raw_chroma, normalized_chroma
    
    except Exception as e:
        print(f"Error visualizing tuning normalization for song {song_id}: {e}")
        return None, None

In [ ]:
# Run the tuning analysis
print("Analyzing tuning variations across the dataset...")
tuning_frequencies, tuning_deviations = analyze_tuning_distribution()

In [ ]:
# Visualize the effect of tuning normalization on an example song
print("\nVisualizing the effect of tuning normalization on an example song...")
raw_chroma, normalized_chroma = visualize_tuning_normalization()

In [ ]:
class ChordDataGenerator(tf.keras.utils.Sequence):
    """Custom data generator for memory-efficient loading"""
    def __init__(self, song_ids, batch_size=32, seq_length=15, step=1, shuffle=True,
                 label_encoder=None, is_training=True, num_classes=None):
        self.song_ids = song_ids
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.step = step
        self.shuffle = shuffle
        self.is_training = is_training
        self.label_encoder = label_encoder
        self.num_classes = num_classes
        
        # Pre-load song lengths to calculate total size
        self.song_data = []
        total_frames = 0
        
        for song_id in tqdm(song_ids, desc="Analyzing songs for generator"):
            chroma = load_chroma(song_id)
            chords = load_chord_labels(song_id)
            
            if chroma is not None and chords is not None:
                # Calculate number of sequences in this song
                num_seqs = max(0, (len(chroma) - seq_length + 1 + step - 1) // step)
                if num_seqs > 0:
                    total_frames += num_seqs
                    self.song_data.append({
                        'song_id': song_id,
                        'num_sequences': num_seqs
                    })
        
        self.total_sequences = total_frames
        print(f"Data generator initialized with {len(self.song_data)} songs and {self.total_sequences} total sequences")
        
        # Create an array of sequence indices for shuffling
        self.indices = np.arange(self.total_sequences)
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def __len__(self):
        """Return the number of batches"""
        return (self.total_sequences + self.batch_size - 1) // self.batch_size
    
    def on_epoch_end(self):
        """Shuffle indices at the end of each epoch if shuffle is True"""
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def __getitem__(self, batch_idx):
        """Generate one batch of data"""
        # Calculate start and end indices for this batch
        batch_start = batch_idx * self.batch_size
        batch_end = min((batch_idx + 1) * self.batch_size, self.total_sequences)
        
        # Get the sequence indices for this batch
        batch_indices = self.indices[batch_start:batch_end]
        
        # Initialize batch arrays
        batch_size = batch_end - batch_start
        X_batch = np.zeros((batch_size, self.seq_length, 12), dtype=np.float32)
        y_batch = np.zeros(batch_size, dtype=np.int32)
        
        # Load the data for this batch
        for i, seq_idx in enumerate(batch_indices):
            # Find which song this sequence belongs to
            song_offset = 0
            for song_data in self.song_data:
                if seq_idx < song_offset + song_data['num_sequences']:
                    song_id = song_data['song_id']
                    sequence_in_song = seq_idx - song_offset
                    
                    # Calculate the starting frame in the song
                    frame_start = sequence_in_song * self.step
                    
                    # Load the song data if not already cached
                    chroma = load_chroma(song_id)
                    chords_data = load_chord_labels(song_id)
                    
                    if chroma is not None and chords_data is not None:
                        # Align chords with chroma
                        aligned_chords = align_chroma_with_chords(chroma, chords_data)
                        
                        # Extract the sequence
                        seq_features = chroma[frame_start:frame_start+self.seq_length]
                        seq_label = aligned_chords[frame_start+self.seq_length-1]
                        
                        # Ensure we have a full sequence (handle edge cases)
                        if len(seq_features) == self.seq_length:
                            X_batch[i] = seq_features
                            
                            # Encode the label
                            if self.label_encoder is not None and seq_label in self.label_encoder.classes_:
                                y_batch[i] = self.label_encoder.transform([seq_label])[0]
                    
                    break
                
                song_offset += song_data['num_sequences']
        
        return X_batch, y_batch

In [ ]:
def prepare_data(song_ids, test_size=0.1, val_size=0.1, seed=42):
    """Prepare the dataset for training, validation, and testing"""
    # Shuffle and split the song IDs
    random.seed(seed)
    song_ids = list(song_ids)
    random.shuffle(song_ids)
    
    # Calculate split indices
    test_idx = int(len(song_ids) * test_size)
    val_idx = int(len(song_ids) * (test_size + val_size))
    
    # Split the song IDs
    test_song_ids = song_ids[:test_idx]
    val_song_ids = song_ids[test_idx:val_idx]
    train_song_ids = song_ids[val_idx:]
    
    print(f"Split {len(song_ids)} songs into {len(train_song_ids)} training, {len(val_song_ids)} validation, and {len(test_song_ids)} test songs")
    
    return train_song_ids, val_song_ids, test_song_ids

In [ ]:
def create_label_encoder(song_ids, max_songs=None):
    """Create a label encoder for the chord labels"""
    all_chords = set()
    processed = 0
    
    for song_id in tqdm(song_ids, desc="Creating label encoder"):
        if max_songs is not None and processed >= max_songs:
            break
            
        chord_data = load_chord_labels(song_id)
        if chord_data is not None:
            all_chords.update(chord_data['chord'].unique())
            processed += 1
    
    print(f"Found {len(all_chords)} unique chords across {processed} songs")
    
    # Create and fit the label encoder
    encoder = LabelEncoder()
    encoder.fit(list(all_chords))
    
    return encoder, list(all_chords)


In [ ]:
# Split the data
train_songs, val_songs, test_songs = prepare_data(common_ids, test_size=TEST_SPLIT, val_size=VALIDATION_SPLIT)

In [ ]:
# Create a label encoder
label_encoder, unique_chords = create_label_encoder(train_songs)
num_classes = len(unique_chords)
print(f"Number of chord classes: {num_classes}")

In [ ]:
# Save the label encoder for later use
with open('models/chord_label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [ ]:
# Create data generators
train_generator = ChordDataGenerator(
    train_songs, 
    batch_size=BATCH_SIZE, 
    seq_length=SEQUENCE_LENGTH, 
    step=STEP_SIZE,
    label_encoder=label_encoder,
    num_classes=num_classes
)

In [ ]:
val_generator = ChordDataGenerator(
    val_songs, 
    batch_size=BATCH_SIZE, 
    seq_length=SEQUENCE_LENGTH, 
    step=STEP_SIZE,
    shuffle=False,
    label_encoder=label_encoder,
    num_classes=num_classes
)

In [ ]:
test_generator = ChordDataGenerator(
    test_songs, 
    batch_size=BATCH_SIZE, 
    seq_length=SEQUENCE_LENGTH, 
    step=STEP_SIZE,
    shuffle=False,
    label_encoder=label_encoder,
    num_classes=num_classes
)

In [ ]:
# Check the most common chord classes
top_chord_counts = Counter()
for song_id in tqdm(train_songs[:50], desc="Analyzing common chords"):
    chord_data = load_chord_labels(song_id)
    if chord_data is not None:
        chord_counts = chord_data['chord'].value_counts().to_dict()
        top_chord_counts.update(chord_counts)

In [ ]:
print("\nTop 20 chord classes in the dataset:")
for chord, count in top_chord_counts.most_common(20):
    chord_idx = label_encoder.transform([chord])[0]
    print(f"{chord} (class {chord_idx}): {count} occurrences")

# 4. Model Development

Let's develop several neural network architectures for chord recognition.

In [ ]:
def create_cnn_model(input_shape, num_classes):
    """Create a CNN model for chord recognition"""
    model = Sequential([
        # Input layer
        Input(shape=input_shape),
        
        # First conv block
        Conv1D(64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        Conv1D(64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        
        # Second conv block
        Conv1D(128, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        Conv1D(128, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        
        # Flatten and dense layers
        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    # Compile the model
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
def create_lstm_model(input_shape, num_classes):
    """Create an LSTM model for chord recognition"""
    model = Sequential([
        # Input layer
        Input(shape=input_shape),
        
        # LSTM layers
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.25),
        Bidirectional(LSTM(128)),
        Dropout(0.25),
        
        # Dense layers
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    # Compile the model
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
def create_hybrid_model(input_shape, num_classes):
    """Create a hybrid CNN-LSTM model for chord recognition"""
    model = Sequential([
        # Input layer
        Input(shape=input_shape),
        
        # CNN feature extraction
        Conv1D(64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        
        # LSTM for temporal dependencies
        Bidirectional(LSTM(128, return_sequences=False)),
        Dropout(0.25),
        
        # Dense layers
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    # Compile the model
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
# Define input shape based on sequence length and features
input_shape = (SEQUENCE_LENGTH, 12)  # 12 chroma features

In [ ]:
# Create models
cnn_model = create_cnn_model(input_shape, num_classes)
lstm_model = create_lstm_model(input_shape, num_classes)
hybrid_model = create_hybrid_model(input_shape, num_classes)

In [ ]:
# Print model summaries
print("CNN Model Summary:")
cnn_model.summary()

In [ ]:
print("\nLSTM Model Summary:")
lstm_model.summary()

In [ ]:
print("\nHybrid CNN-LSTM Model Summary:")
hybrid_model.summary()

# 5. Training and Evaluation

In [ ]:
# Define callbacks
def get_callbacks(model_name):
    return [
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        ),
        ModelCheckpoint(
            filepath=f'models/{model_name}_best.h5',
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        )
    ]

In [ ]:
# Train the CNN model
print("Training CNN Model...")
cnn_history = cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=get_callbacks('cnn'),
    verbose=1
)

In [ ]:
# Train the LSTM model
print("Training LSTM Model...")
lstm_history = lstm_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=get_callbacks('lstm'),
    verbose=1
)

In [ ]:
# Train the hybrid model
print("Training Hybrid CNN-LSTM Model...")
hybrid_history = hybrid_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=get_callbacks('hybrid'),
    verbose=1
)

In [ ]:
# Plot training histories
def plot_history(histories, names):
    plt.figure(figsize=(12, 5))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    for history, name in zip(histories, names):
        plt.plot(history.history['accuracy'], label=f'{name} Train')
        plt.plot(history.history['val_accuracy'], label=f'{name} Val')
    
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    # Plot loss
    plt.subplot(1, 2, 2)
    for history, name in zip(histories, names):
        plt.plot(history.history['loss'], label=f'{name} Train')
        plt.plot(history.history['val_loss'], label=f'{name} Val')
    
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot the training histories
plot_history(
    [cnn_history, lstm_history, hybrid_history],
    ['CNN', 'LSTM', 'Hybrid']
)

In [ ]:
# Evaluate the importance of tuning normalization 
# by comparing performance with and without it
def compare_tuning_normalization_impact():
    """Compare model performance with and without tuning normalization"""
    # We'll use a smaller subset of data for this experiment
    print("Comparing the impact of tuning normalization...")
    
    # First, let's modify the data generator to allow disabling tuning normalization
    class TuningExperimentGenerator(tf.keras.utils.Sequence):
        """Custom generator that can toggle tuning normalization"""
        def __init__(self, song_ids, batch_size=32, seq_length=15, apply_normalization=True,
                     label_encoder=None, shuffle=True, max_songs=50):
            self.song_ids = song_ids[:max_songs]  # Limit number of songs for quick experiment
            self.batch_size = batch_size
            self.seq_length = seq_length
            self.apply_normalization = apply_normalization
            self.label_encoder = label_encoder
            self.shuffle = shuffle
            
            # Pre-load data for this experiment
            self.all_sequences = []
            self.all_labels = []
            
            for song_id in tqdm(self.song_ids, desc=f"Loading data (norm={apply_normalization})"):
                # Load chroma without normalization first
                chroma_path = chordino_dir / song_id / 'bothchroma.csv'
                if not chroma_path.exists():
                    continue
                
                try:
                    # Read the file
                    data = pd.read_csv(chroma_path, header=None)
                    
                    # Extract chroma values (simplified version)
                    numeric_data = data.apply(pd.to_numeric, errors='coerce')
                    chroma_values = None
                    
                    # Try to find chroma columns
                    for i in range(len(data.columns) - 11):
                        test_cols = data.iloc[:, i:i+12]
                        if test_cols.apply(pd.to_numeric, errors='coerce').count().min() > 0.8 * len(data):
                            chroma_values = test_cols.apply(pd.to_numeric, errors='coerce').values
                            break
                    
                    if chroma_values is None:
                        chroma_values = data.iloc[:, 2:14].apply(pd.to_numeric, errors='coerce').values
                    
                    # Replace NaNs with zeros
                    chroma_values = np.nan_to_num(chroma_values)
                    
                    # Apply normalization if specified
                    if self.apply_normalization:
                        chroma_values = normalize_chroma_tuning(chroma_values, song_id)
                    
                    # Load chord labels
                    lab_path = lab_dir / song_id / "full.lab"
                    if not lab_path.exists():
                        continue
                    
                    chord_data = pd.read_csv(lab_path, sep='\t', header=None, names=['start_time', 'end_time', 'chord'])
                    
                    # Align chords with chroma
                    aligned_chords = align_chroma_with_chords(chroma_values, chord_data)
                    
                    # Create sequences
                    for i in range(0, len(chroma_values) - self.seq_length + 1, 10):  # Use a larger step size for efficiency
                        self.all_sequences.append(chroma_values[i:i+self.seq_length])
                        chord = aligned_chords[i+self.seq_length-1]
                        
                        # Encode the chord label
                        if self.label_encoder is not None and chord in self.label_encoder.classes_:
                            self.all_labels.append(self.label_encoder.transform([chord])[0])
                        else:
                            # Skip this sequence if label can't be encoded
                            self.all_sequences.pop()
                
                except Exception as e:
                    print(f"Error processing song {song_id}: {e}")
            
            self.all_sequences = np.array(self.all_sequences)
            self.all_labels = np.array(self.all_labels)
            
            # Create indices for shuffling
            self.indices = np.arange(len(self.all_sequences))
            if self.shuffle:
                np.random.shuffle(self.indices)
            
            print(f"Loaded {len(self.all_sequences)} sequences with normalization={self.apply_normalization}")
        
        def __len__(self):
            return (len(self.all_sequences) + self.batch_size - 1) // self.batch_size
        
        def __getitem__(self, idx):
            # Get indices for this batch
            batch_indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
            return self.all_sequences[batch_indices], self.all_labels[batch_indices]
        
        def on_epoch_end(self):
            if self.shuffle:
                np.random.shuffle(self.indices)
    
    # Create generators with and without normalization
    with_norm_gen = TuningExperimentGenerator(
        song_ids=common_ids,
        batch_size=BATCH_SIZE,
        seq_length=SEQUENCE_LENGTH,
        apply_normalization=True,
        label_encoder=label_encoder,
        shuffle=True
    )
    
    without_norm_gen = TuningExperimentGenerator(
        song_ids=common_ids,
        batch_size=BATCH_SIZE,
        seq_length=SEQUENCE_LENGTH,
        apply_normalization=False,
        label_encoder=label_encoder,
        shuffle=True
    )
    
    # Create a simple model for quick evaluation
    simple_model = Sequential([
        Input(shape=(SEQUENCE_LENGTH, 12)),
        Conv1D(64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    simple_model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Train and evaluate with normalization
    print("\nTraining with tuning normalization:")
    with_norm_history = simple_model.fit(
        with_norm_gen,
        epochs=10,
        verbose=1
    )
    
    # Reset model for fair comparison
    simple_model.set_weights([np.random.normal(0, 0.01, w.shape) for w in simple_model.get_weights()])
    
    # Train and evaluate without normalization
    print("\nTraining without tuning normalization:")
    without_norm_history = simple_model.fit(
        without_norm_gen,
        epochs=10,
        verbose=1
    )
    
    # Plot comparison
    plt.figure(figsize=(12, 5))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(with_norm_history.history['accuracy'], label='With Tuning Normalization')
    plt.plot(without_norm_history.history['accuracy'], label='Without Tuning Normalization')
    plt.title('Effect of Tuning Normalization on Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(with_norm_history.history['loss'], label='With Tuning Normalization')
    plt.plot(without_norm_history.history['loss'], label='Without Tuning Normalization')
    plt.title('Effect of Tuning Normalization on Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    plt.tight_layout()
    plt.show()
    
    # Print final metrics
    print(f"\nFinal accuracy with tuning normalization: {with_norm_history.history['accuracy'][-1]:.4f}")
    print(f"Final accuracy without tuning normalization: {without_norm_history.history['accuracy'][-1]:.4f}")
    print(f"Improvement from tuning normalization: {(with_norm_history.history['accuracy'][-1] - without_norm_history.history['accuracy'][-1])*100:.2f}%")

In [ ]:
# Run the comparison experiment
compare_tuning_normalization_impact()

In [ ]:
# Evaluate the models on the test set
print("Evaluating Models on Test Set...")
cnn_test_results = cnn_model.evaluate(test_generator, verbose=1)
lstm_test_results = lstm_model.evaluate(test_generator, verbose=1)
hybrid_test_results = hybrid_model.evaluate(test_generator, verbose=1)

In [ ]:
print(f"CNN Test Loss: {cnn_test_results[0]:.4f}, Test Accuracy: {cnn_test_results[1]:.4f}")
print(f"LSTM Test Loss: {lstm_test_results[0]:.4f}, Test Accuracy: {lstm_test_results[1]:.4f}")
print(f"Hybrid Test Loss: {hybrid_test_results[0]:.4f}, Test Accuracy: {hybrid_test_results[1]:.4f}")

In [ ]:
# Save the models
cnn_model.save('models/cnn_model.h5')
lstm_model.save('models/lstm_model.h5')
hybrid_model.save('models/hybrid_model.h5')

# 6. Results and Analysis

Let's evaluate the models in more depth and generate predictions.

In [ ]:
def analyze_model_predictions(model, generator, label_encoder, num_batches=10, class_threshold=20):
    """Analyze model predictions on a sample of data"""
    # Generate predictions for a subset of the data
    y_true = []
    y_pred = []
    
    for i in range(min(num_batches, len(generator))):
        X_batch, y_batch = generator[i]
        batch_preds = model.predict(X_batch, verbose=0)
        batch_pred_classes = np.argmax(batch_preds, axis=1)
        
        y_true.extend(y_batch)
        y_pred.extend(batch_pred_classes)
    
    # Convert to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Get the class names
    class_names = label_encoder.classes_
    
    # Calculate overall accuracy
    accuracy = np.mean(y_true == y_pred)
    print(f"Overall accuracy on {len(y_true)} samples: {accuracy:.4f}")
    
    # Identify the most common chords in the test set
    chord_counts = Counter(y_true)
    most_common_chords = chord_counts.most_common()
    
    # Filter to chords with at least class_threshold occurrences
    common_chords = [(chord, count) for chord, count in most_common_chords if count >= class_threshold]
    print(f"Found {len(common_chords)} chord classes with at least {class_threshold} occurrences")
    
    # Calculate per-chord accuracy for common chords
    chord_accuracies = {}
    for chord_idx, count in common_chords:
        mask = (y_true == chord_idx)
        if np.sum(mask) > 0:
            chord_acc = np.mean(y_pred[mask] == chord_idx)
            chord_name = class_names[chord_idx]
            chord_accuracies[chord_name] = (chord_acc, count)
    
    # Plot chord-specific accuracies
    plt.figure(figsize=(14, 8))
    chords = []
    accs = []
    counts = []
    
    for chord, (acc, count) in sorted(chord_accuracies.items(), key=lambda x: x[1][0], reverse=True):
        chords.append(chord)
        accs.append(acc)
        counts.append(count)
    
    # Create a colormap based on accuracies
    colors = plt.cm.viridis(np.array(accs))
    
    ax = plt.bar(range(len(chords)), accs, color=colors)
    plt.xticks(range(len(chords)), chords, rotation=90)
    plt.ylim(0, 1.05)
    plt.xlabel('Chord')
    plt.ylabel('Accuracy')
    plt.title(f'Per-Chord Accuracy (for chords with at least {class_threshold} occurrences)')
    plt.grid(axis='y', linestyle='--', alpha=0.3)
    
    # Add a colorbar
    sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(0, 1))
    sm.set_array([])
    cbar = plt.colorbar(sm)
    cbar.set_label('Accuracy')
    
    plt.tight_layout()
    plt.show()
    
    # Create confusion matrix for top N most common chords
    N = min(15, len(common_chords))
    top_n_chords = [chord for chord, _ in common_chords[:N]]
    
    # Filter to samples with these chords
    mask = np.isin(y_true, top_n_chords)
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    
    # Create confusion matrix
    cm = confusion_matrix(y_true_filtered, y_pred_filtered, labels=top_n_chords)
    
    # Plot confusion matrix
    plt.figure(figsize=(12, 10))
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Replace NaN with 0
    cm_normalized = np.nan_to_num(cm_normalized)
    
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
                xticklabels=[class_names[idx] for idx in top_n_chords],
                yticklabels=[class_names[idx] for idx in top_n_chords])
    plt.xlabel('Predicted Chord')
    plt.ylabel('True Chord')
    plt.title(f'Confusion Matrix (Top {N} Most Common Chords)')
    plt.tight_layout()
    plt.show()
    
    return accuracy, chord_accuracies

In [ ]:
# Choose the best model based on validation accuracy
# For this example, let's analyze the hybrid model
print("Analyzing Hybrid Model Predictions...")
hybrid_accuracy, hybrid_chord_accuracies = analyze_model_predictions(
    hybrid_model, test_generator, label_encoder, num_batches=20, class_threshold=30
)

In [ ]:
def analyze_tuning_impact_by_genre():
    """Analyze if tuning variations correlate with certain genres or eras"""
    # Load the tuning data with song IDs
    tuning_data = []
    for song_id in tqdm(common_ids, desc="Loading tuning data"):
        tuning_path = chordino_dir / song_id / 'tuning.csv'
        if tuning_path.exists():
            try:
                tuning_df = pd.read_csv(tuning_path, header=None)
                tuning_freq = float(tuning_df.iloc[0, 3])
                deviation = 1200 * np.log2(tuning_freq / 440.0)
                
                # Get metadata for this song
                song_meta = index_df[index_df['id'] == int(song_id)].iloc[0]
                
                tuning_data.append({
                    'song_id': song_id,
                    'title': song_meta['title'],
                    'artist': song_meta['artist'],
                    'year': song_meta['year'],
                    'decade': song_meta['decade'],
                    'tuning_freq': tuning_freq,
                    'deviation_cents': deviation
                })
            except Exception as e:
                print(f"Error processing tuning for song {song_id}: {e}")
    
    # Convert to DataFrame
    tuning_df = pd.DataFrame(tuning_data)
    
    # Analyze tuning by decade
    plt.figure(figsize=(14, 8))
    
    # Create a box plot of tuning deviations by decade
    plt.subplot(211)
    sns.boxplot(x='decade', y='deviation_cents', data=tuning_df)
    plt.title('Tuning Deviations by Decade')
    plt.xlabel('Decade')
    plt.ylabel('Deviation (cents)')
    plt.grid(True, linestyle='--', alpha=0.3)
    
    # Create a scatter plot of tuning deviations over time
    plt.subplot(212)
    plt.scatter(tuning_df['year'], tuning_df['deviation_cents'], 
                alpha=0.6, c=np.abs(tuning_df['deviation_cents']), cmap='viridis')
    plt.colorbar(label='Absolute Deviation (cents)')
    plt.title('Tuning Deviations Over Time')
    plt.xlabel('Year')
    plt.ylabel('Deviation (cents)')
    plt.grid(True, linestyle='--', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate average absolute deviation by decade
    decade_stats = tuning_df.groupby('decade')['deviation_cents'].agg(['mean', 'median', 'std', lambda x: np.mean(np.abs(x))])
    decade_stats = decade_stats.rename(columns={'<lambda_0>': 'mean_abs'})
    print("Tuning deviation statistics by decade:")
    print(decade_stats)
    
    # Check if any decades have significantly more tuning variation
    
    decade_groups = [tuning_df[tuning_df['decade'] == decade]['deviation_cents'].values 
                    for decade in tuning_df['decade'].unique()]
    f_stat, p_value = f_oneway(*decade_groups)
    
    print(f"\nANOVA test for differences between decades:")
    print(f"F-statistic: {f_stat:.4f}, p-value: {p_value:.4f}")
    
    if p_value < 0.05:
        print("There are statistically significant differences in tuning between decades.")
    else:
        print("No statistically significant differences in tuning between decades.")
    
    return tuning_df

In [ ]:
# Analyze tuning variations by decade
tuning_by_genre_df = analyze_tuning_impact_by_genre()

# 7. Conclusion and Future Work

In [ ]:
def chord_complexity_analysis(chord_accuracies, label_encoder):
    """Analyze how chord complexity affects recognition accuracy"""
    # Define chord complexity categories
    def get_complexity(chord):
        if chord == 'N':  # No chord
            return 'No Chord'
        elif ':' not in chord:  # Unknown format
            return 'Other'
        
        # Basic classification
        root, quality = chord.split(':', 1)
        
        if quality in ['maj', '']:
            return 'Major'
        elif quality == 'min':
            return 'Minor'
        elif quality in ['7', 'maj7', 'min7']:
            return 'Seventh'
        elif '/' in quality:  # Slash chords (e.g., C:maj/3)
            return 'Slash'
        elif any(ext in quality for ext in ['9', '11', '13']):
            return 'Extended'
        elif any(mod in quality for ext in ['dim', 'aug', 'sus']):
            return 'Altered'
        else:
            return 'Other'
    
    # Group chords by complexity
    complexity_groups = {}
    
    for chord, (acc, count) in chord_accuracies.items():
        complexity = get_complexity(chord)
        if complexity not in complexity_groups:
            complexity_groups[complexity] = []
        complexity_groups[complexity].append((chord, acc, count))
    
    # Calculate average accuracy per complexity group
    avg_accuracies = {}
    for complexity, chords in complexity_groups.items():
        if chords:
            total_samples = sum(count for _, _, count in chords)
            weighted_acc = sum(acc * count for _, acc, count in chords) / total_samples
            avg_accuracies[complexity] = (weighted_acc, total_samples, len(chords))
    
    # Plot results
    plt.figure(figsize=(12, 6))
    
    complexities = []
    accs = []
    counts = []
    num_chords = []
    
    for complexity, (acc, count, n_chords) in sorted(avg_accuracies.items()):
        complexities.append(complexity)
        accs.append(acc)
        counts.append(count)
        num_chords.append(n_chords)
    
    bars = plt.bar(complexities, accs, color='skyblue')
    
    # Add count and number of chords as text
    for i, (bar, count, n_chords) in enumerate(zip(bars, counts, num_chords)):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{count} samples\n{n_chords} chords',
                ha='center', va='bottom', fontsize=9)
    
    plt.ylim(0, 1.05)
    plt.xlabel('Chord Complexity')
    plt.ylabel('Average Accuracy')
    plt.title('Recognition Accuracy by Chord Complexity')
    plt.grid(axis='y', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return avg_accuracies

In [ ]:
# Analyze chord complexity
complexity_results = chord_complexity_analysis(hybrid_chord_accuracies, label_encoder)

In [ ]:
# Final accuracy comparison
model_accuracies = {
    'CNN': cnn_test_results[1],
    'LSTM': lstm_test_results[1],
    'Hybrid CNN-LSTM': hybrid_test_results[1]
}

In [ ]:
plt.figure(figsize=(10, 6))
bars = plt.bar(model_accuracies.keys(), model_accuracies.values(), color=['cornflowerblue', 'lightseagreen', 'coral'])

In [ ]:
# Add accuracy values on top of bars
for bar in bars:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{bar.get_height():.4f}',
            ha='center', va='bottom')

In [ ]:
plt.ylim(0, max(model_accuracies.values()) + 0.1)
plt.xlabel('Model Architecture')
plt.ylabel('Test Accuracy')
plt.title('Model Performance Comparison')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

# Conclusion

In this project, we developed deep learning models for automatic chord recognition using the McGill Billboard dataset. Our approach addressed key challenges including:

1. **Memory Efficiency**: We implemented a custom data generator to handle the large dataset without running into memory limitations, avoiding the need for one-hot encoding of labels.

2. **Temporal Context**: We incorporated sequence modeling to capture the temporal dependencies in music by using windows of consecutive frames.

3. **Model Architecture Comparison**: We compared three different architectures (CNN, LSTM, and hybrid CNN-LSTM) to understand which approach works best for chord recognition.

4. **Chord Complexity Analysis**: We analyzed how recognition accuracy varies with chord complexity, providing insights into the strengths and limitations of our models.

## Key Findings:

- The hybrid CNN-LSTM model achieved the best overall performance, likely due to its ability to capture both local patterns in chroma features and temporal dependencies.
- Simple chords (major and minor) were recognized more accurately than complex chords (extended, altered, and slash chords).
- Using context windows significantly